In [1]:
!pip install  langchain
!pip install  langchain_community
!pip install  pypdf -U
!pip install  transformers==4.38.2
!pip install  tokenizers==0.15.2
!pip install  chromadb
!pip install  langchain_chroma
!pip install  rapidocr-onnxruntime
!pip install  langchain_cohere
!pip install  cohere
!pip install  sentence-transformers


#Import dependencies
import re
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import chromadb
from chromadb.config import Settings
import uuid
from cohere import Client
import warnings
warnings.filterwarnings(action='ignore')
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread
from typing import Optional
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Cohere
from langchain.llms.base import LLM
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 83.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Unin

In [ ]:
def load_document(data_path):
  loader = PyPDFDirectoryLoader(path = data_path, glob='**/[!.]*.pdf', extract_images=True)
  documents = loader.load()
  return documents

def split_document(documents):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
  chunks = text_splitter.split_documents(documents)   #split documents into chunks
  return chunks

In [ ]:
docs = load_document("/content/drive/My Drive/Corpus Legislative/Front_populaire/")
docs_2 = load_document("/content/drive/My Drive/Corpus Legislative/UDD/")
docs_3 = load_document("/content/drive/My Drive/Corpus Legislative/Renaissance/")
docs_Darby = load_document("/content/drive/My Drive/Série principale des lettres de Darby/")

In [ ]:
docs = split_document(docs)
docs_2 = split_document(docs_2)
docs_3 = split_document(docs_3)
docs_Darby = split_document(docs_Darby)

In [ ]:
print(len(docs))

87


In [ ]:
docs[56].page_content

'17\nNouveau Front Populaire \nContrat de législaturePartie 3 Les mois suivants LES TRANSFORMATIONS• Taxation renforcée des transactions financières\n• Créer un droit de préemption pour permettre aux salariés de \nreprendre leur entreprise sous la forme d’une coopérative\n• Accompagner les reprises des entreprises en SCOP par les \nsalariés\n• Créer un pôle public bancaire s’appuyant sur la caisse des dépôts \net des consignations et la banque publique d’investissement qui'

In [ ]:
"""def init_chroma(data:list):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    database = Chroma.from_documents(data,embeddings)
    return database


vector_database = init_chroma(docs)
vector_database_2 = init_chroma(docs_2)
vector_database_3 = init_chroma(docs_3)
vector_database_Darby = init_chroma(docs_Darby)"""

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def init_chroma(data:list, entier:int):
  current_date = datetime.now().strftime("%Y%m%d%H%M%S")
  collection_name = f"chromadb_{current_date}{entier}"
  try:
    chroma_client = chromadb.Client()
    collection = chroma_client.create_collection(
       name=collection_name,
       metadata={"hnsw:space": "cosine"}
       )
    for doc in data:
      collection.add(
         ids=[str(uuid.uuid1())], metadatas=doc.metadata, documents=doc.page_content
      )
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    database = Chroma(
       client = chroma_client,
       collection_name = collection_name,
       embedding_function=embeddings
       )
    return database, collection
  except Exception as e:
    print(f"An error occured: {e}")
    return None, None


vector_database, collection = init_chroma(docs, 0)
vector_database_2, collection_2 = init_chroma(docs_2, 1)
vector_database_3, collection_3 = init_chroma(docs_3, 2)
vector_database_Darby, collection_Darby = init_chroma(docs_Darby, 3)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 40.7MiB/s]


In [ ]:
#Tester s'il y a vraiment un document dans la base de donnée
def verify_documents_in_collection(collection):
    try:
        # Assuming there is a method to count or retrieve documents
        # This may vary based on the actual API of ChromaDB
        document_count = collection.count()
        if document_count > 0:
            print(f"Collection '{collection.name}' contains {document_count} documents.")
        else:
            print(f"Collection '{collection.name}' is empty.")
    except Exception as e:
        print(f"An error occurred while verifying the collection: {e}")

In [ ]:
verify_documents_in_collection(collection_Darby)

Collection 'chromadb_202406202202093' contains 4998 documents.


In [ ]:
def retrieval(database):
  retriever = database.as_retriever(
     search_type="similarity_score_threshold",
     search_kwargs={
        "k": 15,                      #nombre des résultats tirés
        "score_threshold": 0.5        #seuil de la recherche, on prend les top 15 des phrases similaires à la question avec un seuil de 50%
      }
  )
  return retriever

retriever = retrieval(vector_database)
retriever_2 = retrieval(vector_database_2)
retriever_3 = retrieval(vector_database_3)
retriever_Darby = retrieval(vector_database_Darby)

In [ ]:
def reranking(retriever):
  cohere_client = Client(api_key='iqcG53EsoG2Ps2nq1eHZzhJeoNQ4NwR8XliswG9i')
  compressor = CohereRerank(cohere_api_key='iqcG53EsoG2Ps2nq1eHZzhJeoNQ4NwR8XliswG9i', top_n = 3) #on prend les top 3 de ces top 15
  compression_retriever = ContextualCompressionRetriever(
     base_compressor=compressor, base_retriever=retriever
  )
  return compression_retriever

compression_retriever = reranking(retriever)
compression_retriever_2 = reranking(retriever_2)
compression_retriever_3 = reranking(retriever_3)
compression_retriever_Darby = reranking(retriever_Darby)

In [ ]:
# fonction pour initialiser tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    #tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [ ]:
# Initilialiser tokenizer & model
model_name = "HuggingFaceH4/zephyr-7b-beta"                               #"EleutherAI/gpt-j-6b"                                                #
tokenizer = initialize_tokenizer(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id
# specifier stop token ids
stop_token_ids = [0]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
def init_model(model, tokenizer):
    class CustomLLM(LLM):

        streamer: Optional[TextIteratorStreamer] = None

        def _call(self, prompt, stop=None, run_manager=None) -> str:
            model.to("cuda")
            # nouveau prompt de notre LLM
            prompt_template = """
            Use the following context elements to answer the question at the end.
            If the context does not contain any relevant information for the question,
            or there is no given context, simply say that you do not know the answer, do not attempt to invent an answer. Do not try to answer an other question.
            Answer like you were Jean-Luc Mélenchon, Gabriel Attal, Jordan Bardella who are candidates in the legislative elections 2024 in France.
            Always respond in the language in which the question is asked."""
            combined_prompt = f"{prompt_template}\n{prompt}"
            self.streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, timeout=5)
            inputs = tokenizer(combined_prompt, return_tensors="pt").to("cuda")
            kwargs = dict(input_ids=inputs["input_ids"],
                          streamer=self.streamer,
                          max_new_tokens=512)

            # Générer le texte et récupérer la sortie générée
            generated_ids = model.generate(**kwargs)
            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            if combined_prompt in generated_text:
                generated_text = generated_text.replace(combined_prompt, '').strip()
            generated_text = re.sub(r'Question:.*?Helpful Answer:.*?(?=\nQuestion:|\n*$)', '', generated_text, flags=re.DOTALL)
            return generated_text.strip()
        @property
        def _llm_type(self) -> str:
            return "custom"

    llm = CustomLLM()
    return llm
llm_for_response=init_model(model,tokenizer)     #LLM personnalisé pour générer la reponse de l'IA

In [ ]:
def init_model_gnose(model, tokenizer):
    class CustomLLM(LLM):

        streamer: Optional[TextIteratorStreamer] = None

        def _call(self, prompt, stop=None, run_manager=None) -> str:
            model.to("cuda")
            # nouveau prompt de notre LLM
            prompt_template = """
            Use the following context elements to answer the question at the end.
            If the context does not contain any relevant information for the question,
            or there is no given context, simply say that you do not know the answer, do not attempt to invent an answer. Do not try to answer an other question.
            Answer like you were John Nelson Darby who is an Anglo-Irish Bible teacher, one of the influential figures among the original
            Plymouth Brethren and the founder of the Exclusive Brethren. Always respond in the language in which the question is asked."""
            combined_prompt = f"{prompt_template}\n{prompt}"
            self.streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, timeout=5)
            inputs = tokenizer(combined_prompt, return_tensors="pt").to("cuda")
            kwargs = dict(input_ids=inputs["input_ids"],
                          streamer=self.streamer,
                          max_new_tokens=512)

            # Générer le texte et récupérer la sortie générée
            generated_ids = model.generate(**kwargs)
            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            if combined_prompt in generated_text:
                generated_text = generated_text.replace(combined_prompt, '').strip()
            generated_text = re.sub(r'Question:.*?Helpful Answer:.*?(?=\nQuestion:|\n*$)', '', generated_text, flags=re.DOTALL)
            return generated_text.strip()
        @property
        def _llm_type(self) -> str:
            return "custom"

    llm = CustomLLM()
    return llm
llm_for_gnose=init_model_gnose(model,tokenizer)     #LLM personnalisé pour générer la reponse de Darby

In [ ]:
def init_model_for_overview(model, tokenizer):
    class CustomLLM(LLM):

        streamer: Optional[TextIteratorStreamer] = None

        def _call(self, prompt, stop=None, run_manager=None) -> str:
            model.to("cuda")
            self.streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, timeout=5)
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
            kwargs = dict(input_ids=inputs["input_ids"],
                          streamer=self.streamer,
                          max_new_tokens=512)

            # Générer le texte et récupérer la sortie générée
            generated_ids = model.generate(**kwargs)
            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            if prompt in generated_text:
                generated_text = generated_text.replace(prompt, '').strip()
            return generated_text.strip()

        @property
        def _llm_type(self) -> str:
            return "custom"

    llm = CustomLLM()
    return llm
llm_for_overview = init_model_for_overview(model, tokenizer)   #LLM personnalisé pour l'aperçu général du document

In [ ]:
def combined_chunk(document:list)-> str:
    if len(document)>20:
        document = document[:21]
        doc = " ".join(chunk.page_content for chunk in document)
    else:
        doc = " ".join(chunk.page_content for chunk in document)
    return doc
    """doc = " ".join(document[i].page_content for i in range(0, 20))
    return doc"""
# Créer un modèle LangChain pour générer un aperçu général
def create_overview():
    prompt_template = """
        En utilisant le document ci-dessous, générez un texte illustrant
        son titre s'il contient un titre, un très brève résumé de ce qu'il raconte et
        une proposition de trois questions auxquelles ce document peut répondre.
        Essayez de toujours générer des phrases complètes dans le texte généré. Répondez en langue avec laquelle le document est écrit.
        Par exemple: Ce document s'intitule (titre du document). Dans ce document, l'auteur parle de (brève résumé du document).
        Voici trois questions auxquelles ce document peut répondre (les trois questions proposées)".


        {text}


        Réponse:
        """
    prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
    overview_chain = ( prompt | llm_for_overview )
    return overview_chain
def generate_overview(document:str, overview_chain)-> str:
    overview = overview_chain.invoke({"text": document})
    return overview
def overview_chain(document:list)-> str:
  combined_document = combined_chunk(document)
  overview_chain = create_overview()
  overview = generate_overview(combined_document, overview_chain)
  return overview

In [ ]:
# build conversational retrieval chain with memory (rag) using langchain
def Response_IA(query: str, chat_history: list, llm_for_response, compression_retriever)-> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False,
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm_for_response,
            retriever=compression_retriever,
            memory=memory,
            get_chat_history=lambda h: h,
        )
        result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
        generated_text = result['answer']
        generated_text = generated_text.strip()
        chat_history.append((query, generated_text))

        return generated_text, chat_history

    except Exception as e:
        chat_history.append((query, e))
        return e, chat_history

In [4]:
def reference_for_retriever(results):          #fonction pour obtenir les références des programmes
  reference = "Vous pouvez retrouver cette information dans notre programme ici: \n"
  nombre_reference = 1
  for result in results:
    split = result.metadata["source"]
    splitted = split.split("/")
    source = splitted[-1]
    reference = reference + nombre_reference + ") " + source + "," + " " + "page" + " " + result.metadata["page"]+1 + "\n"
    nombre_reference += 1
  return reference

In [ ]:
def reference_for_Darby(results):               #fonction pour obtenir les références de Darby
  nombre_reference = 1
  for result in results:
    reference = nombre_reference + ") '" + result.page_content + "' \n"
    split = result.metadata["source"]
    splitted = split.split("/")
    source = splitted[-1]
    reference = reference + "Source: " + " '" + source,"' dans la page " + result.metadata["page"]+1 + "\n"
    nombre_reference += 1
    return reference

In [ ]:
question_Darby = "Dieu est-il bon d'après Darby?"

In [ ]:
chat_history = []
reponse_Darby, chat_history = Response_IA(question_Darby, chat_history,llm_for_gnose, compression_retriever_Darby)
print(reponse_Darby)

According to the context provided, Darby's belief in God's goodness is not explicitly mentioned. However, the author's statements about God's blessings, mercies, and grace suggest that they share Darby's belief in God's goodness. Therefore, it can be inferred that Darby also believed in God's goodness based on his teachings and writings.


In [ ]:
reference_Darby = compression_retriever_Darby.invoke(question_Darby)
ref_Darby = reference_for_Darby(reference_Darby)
print(ref_Darby)

1 ) ' bien, tout va mieux. Que Dieu daigne bénir pour toute votre famille cette peine de
coeur si réelle, mais où Dieu, toujours bon, a mêlé tant de ménagements et de
grâce à l’amertume de la coupe. Je vous envoie une petite lettre pour Marie; j’ai
eu peur qu’elle ne fût trop longue, mais je suis sûr que, par la bonté de Dieu, elle
jouira de ce petit mot en le lisant à loisir et comme ses forces le lui permettront.
Elle pensera à Christ et sera rafraîchie. '
Source:  ' Lettre de J.N.D. no 459 – Le Messager Evangélique.pdf ' dans la page 2
2 ) ' mieux.
Que Dieu daigne bénir pour toute votre famille cette peine de coeur si réelle, mais
où Dieu toujours plein de bonté a mêlé tant de ménagements et de grâces à
l’amertume de la coupe.
Je vous envoie une petite lettre pour M., j’ai craint de la faire trop longue, mais je
suis sûr que, par la bonté de Dieu, elle jouira de ce petit moment en la lisant à son
loisir et comme sa force le lui permettra. Elle pensera à Christ et, sera rafraîchie. 

In [ ]:
overview = overview_chain(docs_Darby)
print(overview)

Ce document s'intitule "Lettre de J.N.D. no 1" et raconte l'histoire d'une lettre écrite par Darby J.N. En 1851 à un certain M. Il explique sa vision de la sacrificature et de la grâce de Dieu envers nous. Il distingue le salut, la rédemption accomplie pour nous placer pour toujours, sans péché, dans la présence d'un Dieu d'amour, d'un Père, et la gloire qui est à la fin de la course en rapport avec l'oeuvre de l'Esprit, de la responsabilité, qui dépend de l'existence d'une relation. Il explique que la sacrificature s'exerce en grâce pour nous, non sur notre demande, mais avant, et que les sacrifices pour le péché sont accomplis dans le lieu très saint, pour la joie de l'amour de Christ.

        Voici trois questions auxquelles ce document peut répondre:
        
        1. Qu'est la sacrificature et comment s'exerce-t-elle en grâce pour nous?
        
        2. Qu'est la relation entre le salut, la rédemption accomplie et la responsabilité?
        
        3. Qu'est la manière dont

In [ ]:
question = "Que pensez de la situation de la France actuelle?"

In [ ]:
chat_history = []
reponse_Front, chat_history = Response_IA(question, chat_history, llm_for_response, compression_retriever)
print(reponse_Front)

Les députés sont particulièrement associés et / ou à l'initiative de 5 paquets législatifs pour amorcer les grandes bifurcations dont le pays a besoin. D'abord, à la suite des mesures d'urgence par décret, la présentation d'une grande loi permet de rattraper et d'améliorer la situation sociale des Français grandement paupérisés par 7 ans de macronisme et 3 ans d'inflation. Nous pensons donc que la situation de la France actuelle est préoccupante et que des mesures urgentes sont nécessaires pour améliorer la vie des Français les plus pauvres. Nous sommes convaincus que ces paquets législatifs seront un pas décisif vers une France plus juste et plus solidaire.


In [ ]:
reference_Front = compression_retriever_3.invoke(question)
ref_Front = reference_for_retriever(reference_Front)
print(ref_Front)

Vous pouvez retrouver cette information dans notre programme ici:
1 ) Programme-Emmanuel-Macron.pdf , page 2
2 ) Programme-Emmanuel-Macron.pdf , page 2
3 ) Programme-Emmanuel-Macron.pdf , page 16


In [ ]:
chat_history = []
reponse_UDD, chat_history = Response_IA(question, chat_history, llm_for_response, compression_retriever_2)
print(reponse_UDD)

Comme candidat aux élections législatives de 2024 en France, je pense que la France actuelle est confrontée à plusieurs défis économiques et politiques. D'une part, la baisse des impôts de production en France contribue à la poursuite de la réduction des impôts européens. D'autre part, la condition de versement des aides structurelles européennes, largement financées par les Français, à l'achat de produits européens tels que les matériels de défense et les infrastructures énergétiques, est nécessaire. De plus, l'intégration de la lutte contre le chômage et la réindustrialisation dans le mandat de la Banque centrale européenne (BCE) et la mise des politiques monétaires au service de l'investissement dans l'économie réelle sont nécessaires. Enfin, il est nécessaire de mettre les nouvelles générations à l'abri de la France que nous aimons. Je suis prêt à diriger le pays, et j'espère que le peuple votera les 10 et 24 avril pour que le peuple gagne. Pour créer des emplois et préserver l'env

In [ ]:
reference_UDD = compression_retriever_2.invoke(question)
ref_UDD = reference_for_retriever(reference_UDD)
print(ref_UDD)

Vous pouvez retrouver cette information dans notre programme ici:
1 ) Programme.pdf , page 6
2 ) Candidat-06-Marine-Le-Pen-Declaration-accessible.pdf , page 2
3 ) Programme.pdf , page 7


In [ ]:
chat_history = []
reponse_RE, chat_history = Response_IA(question, chat_history, llm_for_response, compression_retriever_3)
print(reponse_RE)

Nous pensons que la France actuelle doit se concentrer sur la réduction de ses déficits budgétaires, en respectant l'engagement pris envers les Français. Nous savons tous que l'État est endetté et que la responsabilité budgétaire est essentielle pour avoir une politique efficace. C'est pourquoi nous nous engagerons à réaliser 60 milliards d'économies en responsabilisant les ministres. Nous souhaitons également que la France répare les injustices de départ et protège chaque citoyen afin de lui permettre de réussir sa vie. Enfin, nous voulons que la France regagne la confiance des Européens et du monde entier en nous redonnant confiance en nous, en la France et dans notre capacité collective à relever nos défis. Nous sommes engagés à régler les problèmes du chômage de masse et de l'intégration depuis plus de trente ans, mais nous reconnaissons que des transformations radicales nouvelles bouleversent nos vies et nos certitudes, en particulier la révolution numérique qui change nos manière

In [ ]:
reference_RE = compression_retriever_3.invoke(question)
ref_RE = reference_for_retriever(reference_RE)
print(ref_RE)

Vous pouvez retrouver cette information dans notre programme ici:
1 ) Programme-Emmanuel-Macron.pdf , page 2
2 ) Programme-Emmanuel-Macron.pdf , page 2
3 ) Programme-Emmanuel-Macron.pdf , page 16
